## 回归 vs 分类
回归是估计一个连续值，预测值y是连续变量  
分类是估计一个离散值，预测值y是离散变量
![vs](./picture/1.png)

Softmax从字面上来说，可以分成soft和max两个部分。max故名思议就是最大值的意思。Softmax的核心在于soft，而soft有软的含义，与之相对的是hard硬。很多场景中需要我们找出数组所有元素中值最大的元素，实质上都是求的hardmax。  
hardmax最大的特点就是只选出其中一个最大的值，即非黑即白。但是往往在实际中这种方式是不合情理的，比如对于文本分类来说，一篇文章或多或少包含着各种主题信息，我们更期望得到文章对于每个可能的文本类别的概率值（置信度），可以简单理解成属于对应类别的可信度。所以此时用到了soft的概念，Softmax的含义就在于不再唯一的确定某一个最大值，而是为每个输出分类的结果都赋予一个概率值，表示属于每个类别的可能性。

类别y是离散值，当y取某一类别的时候在计算机中该怎么表示呢？  
离散特征的编码分为两种情况：

1、离散特征的取值之间没有大小的意义，比如color：[red,blue],那么就使用one-hot编码

2、离散特征的取值有大小的意义，比如size:[X,XL,XXL],那么就使用数值的映射{X:1,XL:2,XXL:3}

## 为什么用独热编码?
独热编码（哑变量 dummy variable）是因为大部分算法是基于向量空间中的度量来进行计算的，为了使非偏序关系的变量取值不具有偏序性，并且到圆点是等距的。使用one-hot编码，将离散特征的取值扩展到了欧式空间，离散特征的某个取值就对应欧式空间的某个点。将离散型特征使用one-hot编码，会让特征各个取值之间的距离计算更加合理。离散特征进行one-hot编码后，编码后的特征，其实每一维度的特征都可以看做是连续的特征。就可以跟对连续型特征的归一化方法一样，对每一维特征进行归一化。比如归一化到[-1,1]或归一化到均值为0,方差为1。

为什么特征向量要映射到欧式空间？

将离散特征通过one-hot编码映射到欧式空间，是因为，在回归，分类，聚类等机器学习算法中，特征之间距离的计算或相似度的计算是非常重要的，而我们常用的距离或相似度的计算都是在欧式空间的相似度计算，计算余弦相似性，基于的就是欧式空间。

比如说color:[red,blue]，对应两个编码分别是(0,1).(1,0);blue和red处于同等地位，不偏向任何一方，在欧式空间中，也是对称的，可互换的，完全对等。  
而size:[X,XL,XXL]，本身蕴含的大小关系，如果用独热编码会导致这种大小关系丢失。

## 网络架构

为了估计所有可能类别的条件概率(根据$\pmb{x}来判断属于各个类别的概率$)，我们需要一个有多个输出的模型，每个类别对应一个输出。
为了解决线性模型的分类问题，我们需要和输出一样多的*仿射函数*（affine function）。
每个输出对应于它自己的仿射函数。
在我们的例子中，由于我们有4个特征和3个可能的输出类别，
我们将需要12个标量来表示权重（带下标的$w$），
3个标量来表示偏置（带下标的$b$）。
下面我们为每个输入计算三个*未规范化的预测*（logit）：$o_1$、$o_2$和$o_3$。

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
$$

我们可以用神经网络图来描述这个计算过程。
与线性回归一样，softmax回归也是一个单层神经网络。
由于计算每个输出$o_1$、$o_2$和$o_3$取决于
所有输入$x_1$、$x_2$、$x_3$和$x_4$，
所以softmax回归的输出层也是全连接层。(全连接层：每个输出取决于所有输入)

![softmax回归是一种单层神经网络](./picture/2.png)

为了更简洁地表达模型，我们仍然使用线性代数符号。
通过向量形式表达为$\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$，
这是一种更适合数学和编写代码的形式。
由此，我们已经将所有权重放到一个$3 \times 4$矩阵中。(3表示有3个输出，4表示每个输出有4个权重，4个输入)
对于给定数据样本的特征$\mathbf{x}$，
我们的输出是由权重与输入特征进行矩阵-向量乘法再加上偏置$\mathbf{b}$得到的。(每个输出对应一个偏置b)

## 规范化
当前模型输出的$\pmb{o}=[o_1,o_2,o_3]$,通过公式得到的$o_1,o_2,o_3 \in [-\infty,+\infty]$,我们想将其映射到$[0,1]$内，且和为1，因为
1. 这样可以将$\pmb{o}$的每一维视为概率.因为概率$\in[0,1]$，且和为1；即$o_1$表示$P(o_1,\pmb x)$,$o_2$表示$P(o_2|\pmb x)$
2. 输出$\pmb{o}$对应欧式空间上的点，与正确类别的独热编码表示对应的欧式空间上的点可以轻松地比较距离，从而容易定义损失函数  
在本章中，将输出$\pmb{o}=[o_1,o_2,o_3]$的每一个分量从$[-\infty,+\infty]$映射到$[0,1]$中的映射函数叫softmax函数，这个映射的过程叫做规范化

原书中对softmax的介绍  
社会科学家邓肯·卢斯于1959年在*选择模型*（choice model）的理论基础上
发明的*softmax函数*正是这样做的：
**softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持
可导的性质。**
为了完成这一目标，我们**首先对每个未规范化的预测求幂，这样可以确保输出非负**。
**为了确保最终输出的概率值总和为1，我们再让每个求幂后的结果除以它们的总和**。如下式：

$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$

这里，对于所有的$j$总有$0 \leq \hat{y}_j \leq 1$。
因此，$\hat{\mathbf{y}}$可以视为一个正确的概率分布。
softmax运算不会改变未规范化的预测$\mathbf{o}$之间的大小次序，只会确定分配给每个类别的概率。
因此，在预测过程中，我们仍然可以用下式来选择最有可能的类别。

$$
\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.
$$

**尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。
因此，softmax回归是一个*线性模型*（linear model）。**

## 小批量样本的矢量化

为了提高计算效率并且充分利用GPU，我们通常会对小批量样本的数据执行矢量计算。
假设我们读取了一个批量的样本$\mathbf{X}$，
其中特征维度（输入数量）为$d$，批量大小为$n$。
此外，假设我们在输出中有$q$个类别。
那么小批量样本的特征为$\mathbf{X} \in \mathbb{R}^{n \times d}$，
权重为$\mathbf{W} \in \mathbb{R}^{d \times q}$，
偏置为$\mathbf{b} \in \mathbb{R}^{1\times q}$。
softmax回归的矢量计算表达式为：

$$ \begin{aligned} \mathbf{O} &= \mathbf{X} \mathbf{W} + \mathbf{b}, \\ \hat{\mathbf{Y}} & = \mathrm{softmax}(\mathbf{O}). \end{aligned} $$

**相对于一次处理一个样本，
小批量样本的矢量化加快了$\mathbf{X}和\mathbf{W}$的矩阵-向量乘法。**
由于$\mathbf{X}$中的每一行代表一个数据样本，
那么softmax运算可以*按行*（rowwise）执行：
对于$\mathbf{O}$的每一行，我们先对所有项进行幂运算，然后通过求和对它们进行标准化。
在 $\mathbf{O} = \mathbf{X} \mathbf{W} + \mathbf{b}$中，
$\mathbf{X} \mathbf{W} + \mathbf{b}$的求和会使用广播机制，
小批量的未规范化预测$\mathbf{O}$和输出概率$\hat{\mathbf{Y}}$
都是形状为$n \times q$的矩阵。

3.1节线性回归中矢量化加速的应用$\uparrow$

## 损失函数
现在我们要计算预测值$\pmb o$与真实值$\pmb y$之间的距离;$\pmb o$的每一维是取该类别的条件概率，$\pmb y$的每一维也是去取该类别的条件概率，但是$\pmb y$只有其正确类别是1，其余维度全为0  
在学习损失函数之前先学习信息论的一些知识。

## 信息论基础
举例说明两个概念：熵和信息量  
某彩票分未中奖，二等奖，一等奖三种，买一张这种彩票未中奖的概率为70%,二等奖的概率为20%，一等奖的概率为10%。假设你是知道这个概率的，用随机变量及其分布来表示，如下表格:

|$X$|0|1|2|
|---|---|---|---|
|p(i)|0.7|0.2|0.1|  

熵的公式$$H[P] = \sum_j - P(j) \log P(j).$$  
通过熵的公式，我们可以计算出*抽彩票*这件事的熵，**熵描述的是这件事情的不确定性**；比如说，如果未中奖的概率为1，其余两项为0，那么抽彩票这件事的结果是非常确定的，如果未中奖，中二等奖，中一等奖的概率平分1，即都是$\frac{1}{3}$,那么抽彩票这件事的结果是最不确定的，因为取哪一个都是等可能的；如果买彩票中奖的概率如表格所示，那么抽彩票这件事的不确定性是比全$\frac{1}{3}$的情况的不确定性要小的  
信息量$\log \frac{1}{P(j)} = -\log P(j)$  
你的朋友买了一张彩票，你觉得这张彩票中了一等奖能让你吃惊，还是这张彩票没中奖让你吃惊？肯定是中了一等奖比没中奖更让你吃惊啊，我们就说中了一等奖的信息量大于没中奖的信息量。  
如果未中奖，中二等奖，中一等奖的概率平分1，即都是$\frac{1}{3}$,然后你的朋友买了一张彩票，你觉得这张彩票中了一等奖能让你吃惊，还是这张彩票没中奖让你吃惊？你会觉得这三种结果发生的概率相同，结果是哪一个都没啥稀奇的，那么就说这三种结果的信息量相同。  
信息量描述*事件结果*的吃惊程度，越低概率的结果发生了，带给你的吃惊程度就越大，该结果所蕴含的信息量就越多。  
熵和信息量的关系  
如果一件事情的结果有很多，每一个的概率都是很小，那么这件事情的不确定性就非常大。每一个结果发生的概率很小，所蕴含的信息量很大，那么这个事件很多个结果的信息量都很大。说明了这件事情的熵大，这件事情的信息量就越大。  
交叉熵从$P$到$Q$，记为$H(P, Q)$。对于某一件事情来说，我们把$P$当作这件事情的主观概率分布律，即我们主观上认为这件事情的分布律；我们把$Q$当作根据一些数据得到的这件事情的概率分布律；那么交叉熵描述的是，**$P$的熵和$Q$的熵的差异**，当$P=Q$时，交叉熵达到最低。在这种情况下，从$P$到$Q$的交叉熵是$H(P, P)= H(P)$。  
比如上面的那个例子，如果我们主观概率分布是表格所示，根据数据得到的概率分布是 $P(0)=P(1)=P(2)=\frac{1}{3}$，计算这两个概率分布的熵的差异，定义$\mathbf{y}=\lbrace 0.7,0.2,0.1\rbrace,\hat{\mathbf{y}}=\lbrace 0.33,0.33,0.33\rbrace$,根据下方交叉熵的公式计算  
$$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$

那么根据上面所学信息论相关内容，我们可以用交叉熵来计算两个概率分布间的特征距离。$$ l(\mathbf{o}, \hat{\mathbf{y}}) = - \sum_{j=1}^q o_j \log \hat{y}_j. $$